# PYMC3 for an ODE system

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import theano
import theano.tensor as T

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import pymc3 as pm

Hack the ODE solver: Use `solve_vip` with solver `RK45` and fixed step of 0.02 (remember that time will run from 0 to 1).

In [3]:
import scipy
import theano

floatX = theano.config.floatX

class DE(pm.ode.DifferentialEquation):
    def _simulate(self, y0, theta):
        # Initial condition comprised of state initial conditions and raveled sensitivity matrix
        s0 = np.concatenate([y0, self._sens_ic])
        
        # perform the integration
        sol = scipy.integrate.solve_ivp(
            fun = lambda t, Y: self._system(Y, t, tuple(np.concatenate([y0, theta]))),
            t_span=[self._augmented_times.min(), self._augmented_times.max()],
            y0=s0,
            method='RK23',
            t_eval=self._augmented_times[1:],
            atol=1, rtol=1,
            max_step=0.02).y.T.astype(floatX)
        
        # The solution
        y = sol[:, :self.n_states]

        # The sensitivities, reshaped to be a sequence of matrices
        sens = sol[0:, self.n_states:].reshape(self.n_times, self.n_states, self.n_p)

        return y, sens

Define some functions:

In [4]:
def ploidy(t):
    t0 = [  0.        ,   2.04081633,   4.08163265,   6.12244898,
         8.16326531,  10.20408163,  12.24489796,  14.28571429,
        16.32653061,  18.36734694,  20.40816327,  22.44897959,
        24.48979592,  26.53061224,  28.57142857,  30.6122449 ,
        32.65306122,  34.69387755,  36.73469388,  38.7755102 ,
        40.81632653,  42.85714286,  44.89795918,  46.93877551,
        48.97959184,  51.02040816,  53.06122449,  55.10204082,
        57.14285714,  59.18367347,  61.2244898 ,  63.26530612,
        65.30612245,  67.34693878,  69.3877551 ,  71.42857143,
        73.46938776,  75.51020408,  77.55102041,  79.59183673,
        81.63265306,  83.67346939,  85.71428571,  87.75510204,
        89.79591837,  91.83673469,  93.87755102,  95.91836735,
        97.95918367, 100.        ]
    C0 = [0.91831285, 0.91831285, 0.9169479 , 0.91488274, 0.91247514,
       0.90972508, 0.90663256, 0.9031976 , 0.89942018, 0.8953003 ,
       0.89083798, 0.8860332 , 0.88088596, 0.87539628, 0.86956414,
       0.86338954, 0.8568725 , 0.850013  , 0.84281104, 0.83526664,
       0.82737978, 0.81915047, 0.8105787 , 0.80166448, 0.79240781,
       0.78280868, 0.7728671 , 0.76258307, 0.75195659, 0.74098765,
       0.72967626, 0.71802241, 0.70602611, 0.69368736, 0.68100616,
       0.6679825 , 0.65461639, 0.64090782, 0.6268568 , 0.61246333,
       0.59772741, 0.58264903, 0.5672282 , 0.55146492, 0.54168115,
       0.54168115, 0.54168115, 0.54168115, 0.54168115, 0.54168115]
    C = 0
    for i in range(len(t0)-1):
        C += (t>=t0[i]) * (t<t0[i+1]) * (C0[i]*(t0[i+1]-t) + C0[i+1]*(t-t0[i]))/(t0[i+1] - t0[i])
    return C

def ploidy2x2(t):
    t0 = [  0.        ,   2.04081633,   4.08163265,   6.12244898,
         8.16326531,  10.20408163,  12.24489796,  14.28571429,
        16.32653061,  18.36734694,  20.40816327,  22.44897959,
        24.48979592,  26.53061224,  28.57142857,  30.6122449 ,
        32.65306122,  34.69387755,  36.73469388,  38.7755102 ,
        40.81632653,  42.85714286,  44.89795918,  46.93877551,
        48.97959184,  51.02040816,  53.06122449,  55.10204082,
        57.14285714,  59.18367347,  61.2244898 ,  63.26530612,
        65.30612245,  67.34693878,  69.3877551 ,  71.42857143,
        73.46938776,  75.51020408,  77.55102041,  79.59183673,
        81.63265306,  83.67346939,  85.71428571,  87.75510204,
        89.79591837,  91.83673469,  93.87755102,  95.91836735,
        97.95918367, 100.        ]
    C0 = [0.03622257, 0.03622257, 0.03910149, 0.04282893, 0.04657066,
       0.0503267 , 0.05409704, 0.05788169, 0.06168063, 0.06549387,
       0.06932142, 0.07316327, 0.07701942, 0.08088987, 0.08477463,
       0.08867368, 0.09258704, 0.0965147 , 0.10045666, 0.10441292,
       0.10838349, 0.11236835, 0.11636752, 0.12038099, 0.12440876,
       0.12845084, 0.13250721, 0.13657789, 0.14066286, 0.14476214,
       0.14887573, 0.15300361, 0.1571458 , 0.16130228, 0.16547307,
       0.16965816, 0.17385755, 0.17807125, 0.18229924, 0.18654154,
       0.19079814, 0.19506904, 0.19935424, 0.20365375, 0.20628347,
       0.20628347, 0.20628347, 0.20628347, 0.20628347, 0.20628347]
    C = 0
    for i in range(len(t0)-1):
        C += (t>=t0[i]) * (t<t0[i+1]) * (C0[i]*(t0[i+1]-t) + C0[i+1]*(t-t0[i]))/(t0[i+1] - t0[i])
    return C

def dtploidy(t):
    t0 = [  0.        ,   2.04081633,   4.08163265,   6.12244898,
         8.16326531,  10.20408163,  12.24489796,  14.28571429,
        16.32653061,  18.36734694,  20.40816327,  22.44897959,
        24.48979592,  26.53061224,  28.57142857,  30.6122449 ,
        32.65306122,  34.69387755,  36.73469388,  38.7755102 ,
        40.81632653,  42.85714286,  44.89795918,  46.93877551,
        48.97959184,  51.02040816,  53.06122449,  55.10204082,
        57.14285714,  59.18367347,  61.2244898 ,  63.26530612,
        65.30612245,  67.34693878,  69.3877551 ,  71.42857143,
        73.46938776,  75.51020408,  77.55102041,  79.59183673,
        81.63265306,  83.67346939,  85.71428571,  87.75510204,
        89.79591837,  91.83673469,  93.87755102,  95.91836735,
        97.95918367, 100.        ]
    C0 = [-0.00079798, -0.00079798, -0.00092802, -0.00109583, -0.00126363,
       -0.00143143, -0.00159923, -0.00176703, -0.00193484, -0.00210264,
       -0.00227044, -0.00243824, -0.00260605, -0.00277385, -0.00294165,
       -0.00310945, -0.00327725, -0.00344506, -0.00361286, -0.00378066,
       -0.00394846, -0.00411626, -0.00428407, -0.00445187, -0.00461967,
       -0.00478747, -0.00495527, -0.00512308, -0.00529088, -0.00545868,
       -0.00562648, -0.00579429, -0.00596209, -0.00612989, -0.00629769,
       -0.00646549, -0.0066333 , -0.0068011 , -0.0069689 , -0.0071367 ,
       -0.0073045 , -0.00747231, -0.00764011, -0.00780791, -0.00791027,
       -0.00791027, -0.00791027, -0.00791027, -0.00791027, -0.00791027]
    C = 0
    for i in range(len(t0)-1):
        C += (t>=t0[i]) * (t<t0[i+1]) * (C0[i]*(t0[i+1]-t) + C0[i+1]*(t-t0[i]))/(t0[i+1] - t0[i])
    return C

def Catm(t):
    t0 = [1900.        , 1902.44897959, 1904.89795918, 1907.34693878,
       1909.79591837, 1912.24489796, 1914.69387755, 1917.14285714,
       1919.59183673, 1922.04081633, 1924.48979592, 1926.93877551,
       1929.3877551 , 1931.83673469, 1934.28571429, 1936.73469388,
       1939.18367347, 1941.63265306, 1944.08163265, 1946.53061224,
       1948.97959184, 1951.42857143, 1953.87755102, 1956.32653061,
       1958.7755102 , 1961.2244898 , 1963.67346939, 1966.12244898,
       1968.57142857, 1971.02040816, 1973.46938776, 1975.91836735,
       1978.36734694, 1980.81632653, 1983.26530612, 1985.71428571,
       1988.16326531, 1990.6122449 , 1993.06122449, 1995.51020408,
       1997.95918367, 2000.40816327, 2002.85714286, 2005.30612245,
       2007.75510204, 2010.20408163, 2012.65306122, 2015.10204082,
       2017.55102041, 2020.        ]
    C0 = [-0.003165  , -0.00398316, -0.00480133, -0.00561796, -0.00643061,
       -0.00724582, -0.00806398, -0.00888214, -0.00970031, -0.01051388,
       -0.01132663, -0.0121448 , -0.01296296, -0.01378112, -0.01459714,
       -0.0154098 , -0.01622561, -0.02009388, -0.02128673, -0.02208469,
       -0.02167551, -0.02478571, -0.02452245, -0.01043776,  0.09151429,
        0.21662551,  0.42091429,  0.78627755,  0.61937857,  0.53691735,
        0.46663469,  0.38783061,  0.33636735,  0.28589898,  0.24261837,
        0.20758571,  0.18546224,  0.16226531,  0.13925204,  0.12203265,
        0.10732857,  0.09074388,  0.07862857,  0.06197444,  0.05349545,
        0.04584843,  0.03678004,  0.01837024,  0.0096852 ,  0.0097    ]
    C = 0
    for i in range(len(t0)-1):
        C += (t>=t0[i]) * (t<t0[i+1]) * (C0[i]*(t0[i+1]-t) + C0[i+1]*(t-t0[i]))/(t0[i+1] - t0[i])
    return C

## Some input data:

In [5]:
Dbirth = np.array([1992.5       , 1958.16666667, 1982.16666667, 1959.33333333,
       1975.33333333, 1963.58333333, 1935.58333333, 1951.5       ,
       1951.33333333, 1947.83333333, 1995.33333333, 1987.33333333,
       1964.        , 1961.58333333, 1935.41666667, 1997.33333333,
       1983.91666667, 1975.33333333, 1982.08333333, 1953.        ,
       1975.33333333, 1952.91666667, 1948.16666667, 1937.58333333,
       1992.16666667, 1937.58333333, 1971.08333333, 1979.83333333,
       1978.83333333, 1942.83333333, 1968.91666667, 1978.83333333,
       1946.5       , 1943.33333333, 1961.58333333, 1975.08333333,
       1961.58333333, 1945.83333333, 1945.83333333, 1945.83333333,
       1961.58333333])
subject_age = np.array([23.83333333, 59.25      , 30.33333333, 58.16666667, 44.41666667,
       54.16666667, 82.75      , 66.        , 66.        , 70.08333333,
       21.41666667, 25.16666667, 52.91666667, 55.83333333, 82.08333333,
       20.        , 33.58333333, 42.16666667, 34.41666667, 64.41666667,
       44.41666667, 64.5       , 69.33333333, 79.08333333, 25.25      ,
       79.08333333, 46.08333333, 37.5       , 33.58333333, 73.58333333,
       43.58333333, 33.58333333, 58.33333333, 73.75      , 50.91666667,
       37.33333333, 50.91666667, 59.        , 59.        , 59.        ,
       50.91666667])
wn2 = np.array([0.78977589, 0.58808306, 0.76092518, 0.59547375, 0.77876372,
       0.62245028, 0.41790875, 0.54063138, 0.54063138, 0.51122149,
       0.79931132, 0.78422512, 0.63075114, 0.61131173, 0.42287748,
       0.80461512, 0.74481353, 0.69769879, 0.74056034, 0.55189679,
       0.        , 0.55130714, 0.51669616, 0.62415201, 0.78386839,
       0.        , 0.67420201, 0.72411102, 0.74481353, 0.48571962,
       0.68933236, 0.81894033, 0.59433442, 0.48448504, 0.75332387,
       0.72502724, 0.64378604, 0.58979334, 0.71983191, 0.        ,
       0.        ])
wn4 = np.array([0.21022411, 0.41191694, 0.23907482, 0.40452625, 0.22123628,
       0.37754972, 0.58209125, 0.45936862, 0.45936862, 0.48877851,
       0.20068868, 0.21577488, 0.36924886, 0.38868827, 0.57712252,
       0.19538488, 0.25518647, 0.30230121, 0.25943966, 0.44810321,
       1.        , 0.44869286, 0.48330384, 0.37584799, 0.21613161,
       1.        , 0.32579799, 0.27588898, 0.25518647, 0.51428038,
       0.31066764, 0.18105967, 0.40566558, 0.51551496, 0.24667613,
       0.27497276, 0.35621396, 0.41020666, 0.28016809, 1.        ,
       1.        ])
n = len(Dbirth)
scaled_age = 1

## Model POP1

### Simulate:

In [6]:
lambda_true = 0.01
sigma_true = 0.001

In [7]:
def ode_C_func_scaled(C, t, p):
    lambda_ = p[0] 
    t *= subject_age / scaled_age
    dC = (subject_age / scaled_age) * lambda_*(Catm(t + Dbirth) - C)
    return [dC[i] for i in range(n)]

In [8]:
%%time
s_exact = sp.integrate.solve_ivp(lambda t, y: ode_C_func_scaled(y, t, tuple([[lambda_true]])),
                                   t_span=[0, scaled_age],
                                   y0=Catm(Dbirth),
                                   method='BDF',
                                   t_eval=[scaled_age]
                          )

CPU times: user 796 ms, sys: 569 ms, total: 1.36 s
Wall time: 203 ms


In [9]:
%%time
s = sp.integrate.solve_ivp(lambda t, y: ode_C_func_scaled(y, t, tuple([[lambda_true]])),
                                   t_span=[0, scaled_age],
                                   y0=Catm(Dbirth),
                                   method='RK23',
                                   t_eval=[scaled_age],
                           min_step=0.02,
                           max_step=0.02,
                           rtol=1, atol=1
                          )

CPU times: user 374 ms, sys: 272 ms, total: 646 ms
Wall time: 129 ms


/home/fabrost/miniconda3/envs/spols191122/lib/python3.7/site-packages/scipy/integrate/_ivp/common.py:41: UserWarning: The following arguments have no effect for a chosen solver: `min_step`.
  .format(", ".join("`{}`".format(x) for x in extraneous)))


In [10]:
np.abs(s_exact.y - s.y).max()

0.00041084014454959017

In [11]:
d14C_obs = np.random.normal(s.y.T[0], sigma_true)

### Inference

In [12]:
%%time
ode_C_model = DE(ode_C_func_scaled, times=[scaled_age], n_states=n, n_theta=1)

CPU times: user 18.4 s, sys: 2.07 s, total: 20.5 s
Wall time: 35.6 s


In [14]:
%%time
with pm.Model() as POP1:
    lambda_ = pm.Uniform('lambda_', 0, 1)
    sigma = pm.HalfNormal('sigma', 0.01)
    
    ode_C_solution = ode_C_model(y0=Catm(Dbirth),
                                 theta=[lambda_]).T.reshape(d14C_obs.shape)
    
    d14C = pm.Normal('d14C', mu=ode_C_solution, sd=sigma, observed=d14C_obs)
#     trace = pm.sample(100, tune=50, chains=1, compute_convergence_checks=False)
    approx = pm.fit(n=1000)

Average Loss = 23,226:  10%|█         | 105/1000 [00:55<07:55,  1.88it/s]
Interrupted at 105 [10%]: Average Loss = 23,147


CPU times: user 59.6 s, sys: 905 ms, total: 1min
Wall time: 1min 7s


## Model D

### Simulate

In [ ]:
r2_true = 0.1
r4_true = 0.02
kappa24_true = 0.01
kappa42_true = 0.001

In [ ]:
def delta2_D(r2, r4, kappa24, kappa42, t):
    return -kappa24 - kappa42 + r2 + kappa42/ploidy(t) - (dtploidy(t)*(1 + ploidy(t))**2)/(2.*ploidy(t))

def delta4_D(r2, r4, kappa24, kappa42, t):
    return -(dtploidy(t)*(1 + ploidy(t))**2 + 4*(-kappa42 + r4 + (2*kappa24 + kappa42 - r4)*ploidy(t)))/(4.*(-1 + ploidy(t)))

In [ ]:
def ode_C2C4_func_scaled(y, t, p):
    t *= (subject_age / scaled_age)
    C2 = y[:n]
    C4 = y[n:]
    
    r2 = p[0]
    r4 = p[1]
    kappa24 = p[2]
    kappa42 = p[3]
    
    delta2 = delta2_D(r2, r4, kappa24, kappa42, t)
    delta4 = delta4_D(r2, r4, kappa24, kappa42, t)
#     assert (delta2 > 0).all()
#     assert (delta4 > 0).all()
    
    dC2 = (subject_age / scaled_age) *(C2*kappa42 - C4*kappa42 - C2*r2 + r2*Catm(Dbirth + t) + ((-C2 + C4)*kappa42)/ploidy(t))
    dC4 = (subject_age / scaled_age) * ((-(C4*(delta4 + kappa42 - 2*r4)) + (-(C2*kappa24) + C4*(delta4 + 4*kappa24 + kappa42 - 2*r4))*ploidy(t) + Catm(Dbirth + t)*(-r4 + (-kappa24 + r4)*ploidy(t)))/(-1 + ploidy(t)))
    return [dC2[i] for i in range(n)] + [dC4[i] for i in range(n)]

In [ ]:
s = sp.integrate.solve_ivp(lambda t, y: ode_C2C4_func_scaled(y, t,
                                                             [r2_true, r4_true,
                                                                     kappa24_true, kappa42_true]),
                                   t_span=[0, scaled_age],
                                   y0=np.concatenate([Catm(Dbirth), Catm(Dbirth)]),
                                   method='RK45',
                                   t_eval=[scaled_age])

In [ ]:
C2 = s.y[:n, 0]
C4 = s.y[n:, 0]
y = wn2*C2 + wn4*C4

np.random.seed(42)
yobs = np.random.normal(loc=y, scale=sigma_true)

### Inference

In [ ]:
%%time
ode_C2C4_model = DE(ode_C2C4_func_scaled, times=[scaled_age], n_states=2*n, n_theta=4)

In [ ]:
%%time
Cdata = yobs

with pm.Model() as D:
    lower = 0
    upper = 1.0
    
    r2 = pm.Uniform('r2', lower, upper, testval=0.1)
    r4 = pm.Uniform('r4', testval=0.1)
    kappa24 = pm.Uniform('kappa24', testval=0.1)
    kappa42 = pm.Uniform('kappa42', testval=0.1)
    
    sigma = pm.Uniform('sigma', lower, upper)
    
    test_times = np.linspace(0, subject_age.max(), 3)
    for t in test_times:
        pm.Potential(f'delta2_{t}',
                     (delta2_D(r2, r4, kappa24, kappa42, t)<0) * 10*delta2_D(r2, r4, kappa24, kappa42, t)**2)
        pm.Potential(f'delta4_{t}',
                     (delta4_D(r2, r4, kappa24, kappa42, t)<0) * 10*delta4_D(r2, r4, kappa24, kappa42, t)**2)
    
    ode_C2C4_solution = ode_C2C4_model(y0=np.concatenate([Catm(Dbirth), Catm(Dbirth)]), 
                                       theta=[r2, r4, kappa24, kappa42])
    
    C2 = ode_C2C4_solution[0, :n]
    C4 = ode_C2C4_solution[0, n:]
    mu = wn2*C2 + wn4*C4
    
    d14C = pm.Normal('d14C', mu=mu, sd=sigma, observed=Cdata)
    trace = pm.sample(100, tune=50, chains=1, compute_convergence_checks=False)